In [25]:
!pip install pandas openpyxl scikit-learn tensorflow

In [26]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow import keras
from tensorflow.keras.layers import Embedding, LSTM, Dense, Input
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [27]:
df = pd.read_excel('/content/8emotionscombineddata .xlsx', engine='openpyxl')

In [29]:
X = df['text'].values
y_emotion = df['emot'].values
y_blf = df['blf2'].values

In [30]:
label_encoder_emotion = LabelEncoder()
label_encoder_blf = LabelEncoder()

In [31]:
y_emotion_encoded = label_encoder_emotion.fit_transform(y_emotion)
y_blf_encoded = label_encoder_blf.fit_transform(y_blf)

In [33]:
max_words = 10000
max_seq_len = 100
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(X)

In [34]:
X_seq = tokenizer.texts_to_sequences(X)
X_padded = pad_sequences(X_seq, maxlen=max_seq_len)

In [35]:
X_train, X_test, y_emotion_train, y_emotion_test, y_blf_train, y_blf_test = train_test_split(
    X_padded, y_emotion_encoded, y_blf_encoded, test_size=0.2, random_state=42)

In [36]:
input_layer = Input(shape=(max_seq_len,))
embedding_layer = Embedding(input_dim=max_words, output_dim=128)(input_layer)
lstm_layer = LSTM(64)(embedding_layer)
emotion_output = Dense(len(label_encoder_emotion.classes_), activation='softmax', name='emotion_output')(lstm_layer)
blf_output = Dense(len(label_encoder_blf.classes_), activation='softmax', name='blf_output')(lstm_layer)

model = Model(inputs=input_layer, outputs=[emotion_output, blf_output])

In [37]:
model.compile(optimizer='adam',
              loss={'emotion_output': 'sparse_categorical_crossentropy', 'blf_output': 'sparse_categorical_crossentropy'},
              loss_weights={'emotion_output': 0.5, 'blf_output': 0.5},
              metrics=['accuracy'])

In [39]:
history = model.fit(X_train, {'emotion_output': y_emotion_train, 'blf_output': y_blf_train},
                    epochs=200, batch_size=64, validation_split=0.2)

Epoch 1/200
17/17 [==============================] - 2s 131ms/step - loss: 2.8575 - emotion_output_loss: 3.7672 - blf_output_loss: 1.9478 - emotion_output_accuracy: 0.1813 - blf_output_accuracy: 0.4269 - val_loss: 3.1778 - val_emotion_output_loss: 4.1193 - val_blf_output_loss: 2.2363 - val_emotion_output_accuracy: 0.1946 - val_blf_output_accuracy: 0.2607
Epoch 2/200
17/17 [==============================] - 2s 101ms/step - loss: 2.7335 - emotion_output_loss: 3.6909 - blf_output_loss: 1.7762 - emotion_output_accuracy: 0.2008 - blf_output_accuracy: 0.4825 - val_loss: 3.1642 - val_emotion_output_loss: 4.0991 - val_blf_output_loss: 2.2294 - val_emotion_output_accuracy: 0.1946 - val_blf_output_accuracy: 0.2802
Epoch 3/200
17/17 [==============================] - 1s 89ms/step - loss: 2.6368 - emotion_output_loss: 3.6310 - blf_output_loss: 1.6425 - emotion_output_accuracy: 0.2135 - blf_output_accuracy: 0.5234 - val_loss: 3.1300 - val_emotion_output_loss: 4.0557 - val_blf_output_loss: 2.2044 - 

In [45]:
results = model.evaluate(X_test, {'emotion_output': y_emotion_test, 'blf_output': y_blf_test})


11/11 [==============================] - 0s 6ms/step - loss: 4.3198 - emotion_output_loss: 5.1794 - blf_output_loss: 3.4601 - emotion_output_accuracy: 0.2804 - blf_output_accuracy: 0.3769


In [46]:
total_loss = results[0]
emotion_accuracy = results[3]
blf_accuracy = results[4]

In [47]:
print(f'Total Loss: {total_loss:.4f}')
print(f'Emotion Accuracy: {emotion_accuracy * 100:.2f}%')
print(f'BLF Accuracy: {blf_accuracy * 100:.2f}%')

Total Loss: 4.3198
Emotion Accuracy: 28.04%
BLF Accuracy: 37.69%


In [48]:
model.save('emotion_blf_model.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [49]:
loaded_model = keras.models.load_model('emotion_blf_model.h5')

In [50]:
new_df = pd.read_excel('/content/testdata.xlsx', engine='openpyxl')
X_new = new_df['text'].values

In [51]:
X_new_seq = tokenizer.texts_to_sequences(X_new)
X_new_padded = pad_sequences(X_new_seq, maxlen=max_seq_len)

In [52]:
emotion_predictions, blf_predictions = loaded_model.predict(X_new_padded)


9/9 [==============================] - 0s 4ms/step


In [53]:
predicted_emotions = label_encoder_emotion.inverse_transform(np.argmax(emotion_predictions, axis=1))
predicted_blf = label_encoder_blf.inverse_transform(np.argmax(blf_predictions, axis=1))


In [54]:
new_df['Predicted_Emotion'] = predicted_emotions
new_df['Predicted_BLF'] = predicted_blf

In [55]:
new_df.to_excel('new_data_with_predictions.xlsx', index=False, engine='openpyxl')